# Обработка входных и выходных данных

Примитив `RunnableParallel` является словарем со значениями в форме Runnable или объектами, которые могут быть приведены к такой форме, например, функцииями.
Он исполняет все свои значения параллельно.
Каждый элемент вызывается с общим входным значением `RunnableParallel`.
Примитив возвращает словарь с результатами выполнения каждого значения под соответствующим ключом.

`RunnableParallel` используется для параллелизации, но его также можно применять для преобразования вывода одного Runnable-объекта во входной формат следующего Runnable-объекта.

Here the input to prompt is expected to be a map with keys "context" and "question". The user input is just the question. So we need to get the context using our retriever and passthrough the user input under the "question" key.

В примере на вход в промпт нужно передавать данные в формате map-структуры с полями `context` и `question`.
Ввод пользователя — это просто вопрос.
Поэтому нужно получить контекст с помощью ретривера и передать ввод пользователя в поле  `question`.



In [ ]:
%pip install --upgrade --quiet  gigachain faiss-cpu

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=GigaChatEmbeddings(credentials="<авторизационные_данные>", verify_ssl_certs=False)
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False, scope="GIGACHAT_API_CORP")

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

:::note

При соединении RunnableParallel с другим Runnable-объектом не нужно оборачивать словарь в класс RunnableParallel, так как типы преобразуются автоматически.
В контексте цепочки оба варианта будут работать одинаково:

:::

```python
{"context": retriever, "question": RunnablePassthrough()}
```

```python
RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
```

```python
RunnableParallel(context=retriever, question=RunnablePassthrough())
```



## Использование itemgetter

При работе с `RunnableParallel` для упрощения извлечения данных из map-структуры можно использовать Python-функцию [`itemgetter`](https://docs.python.org/3/library/operator.html#operator.itemgetter).


Пример ниже демонстрирует, как извлечь определенные поля из map-структуры:

In [6]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=GigaChatEmbeddings(credentials="<авторизационные_данные>", verify_ssl_certs=False)
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "italian"})

'Harrison ha lavorato a Kensho.'

## Параллелизация выполнения

`RunnableParallel` позволяет параллельно выполнять несколько Runnable-объектов и возвращать результат их работы в виде map-структуры.

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain.chat_models.gigachat import GigaChat

model = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False)
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!"),
 'poem': AIMessage(content="In the wild's embrace, bear roams free,\nStrength and grace, a majestic decree.")}

## Параллелизм

`RunnableParallel` также полезен для одновременного выполнения независимых процессов, поскольку каждый экземпляр Runnable в map-структуре выполняется параллельно.
Так, на примере представленных цепочек `joke_chain`, `poem_chain` и `map_chain` можно убедиться, что все они выполняются примерно за одно время.
При этом цепочка `map_chain` выполняет две другие цепочки.

In [6]:
%%timeit

joke_chain.invoke({"topic": "bear"})

958 ms ± 402 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit

poem_chain.invoke({"topic": "bear"})

1.22 s ± 508 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit

map_chain.invoke({"topic": "bear"})

1.15 s ± 119 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
